In [ ]:
import numpy as np
import pandas as pd

import utils
from sklearn.metrics.pairwise import paired_distances
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns 



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotnine as pn

from sklearn.metrics.pairwise import cosine_similarity, paired_distances
from sklearn.preprocessing import StandardScaler, scale
from scipy.spatial import distance

from einops import rearrange, reduce, repeat


In [ ]:
import importlib
importlib.reload(utils)


In [ ]:
subjs = utils.ExpInfo.getSubjIDs()
task = utils.ExpInfo.taskName[0]


# Group analysis

In [ ]:
wSize = 60

def group(subj):
    df_beh = utils.LoadData.behaviorData(subj, task)
    # _, h, _ = utils.LoadData.xhy(subj, task, wSize=wSize)
    # _, h_disp, _ = utils.LoadData.xhy_disp(subj, task, wSize=wSize)
    h, h_disp = utils.LoadData.mouseMovement_array(subj, task, velocity=True, packDot=True)
    dist_measure = 'cosine'

    # dist_timeSeries = []
    # v_timeSeries = []
    d = []
    for iTrial in range(len(h)):
        h_trial = h[iTrial]
        h_disp_trial = h_disp[iTrial]

        # hidden action plan velocity
        vh = np.diff(h_trial, axis=0)
        vh_disp_dot = np.diff(h_disp_trial, axis=0)

        dist_timeSeries_ = paired_distances(h_trial, h_disp_trial, metric=dist_measure)
        v_timeSeries_ = paired_distances(vh, vh_disp_dot, metric=dist_measure)
        
        d.append({'dist_mean': dist_timeSeries_.mean(), 
                  'dist_sd': dist_timeSeries_.std(),
                  'v_mean': v_timeSeries_.mean(),
                  'v_sd': v_timeSeries_.std()})
        
        # change  distance to "similarity" index
        # d.append({'dist_mean': 1-dist_timeSeries_.mean(), 
        #         'dist_sd': 1-dist_timeSeries_.std(),
        #         'v_mean': 1-v_timeSeries_.mean(),
        #         'v_sd': 1-v_timeSeries_.std()})    
        
    df_ = pd.concat([df_beh, pd.DataFrame(d)], axis=1)
    X = df_[['dist_mean', 'v_mean']]
    # X = StandardScaler().fit_transform(X)

    y = df_['response']
    LR = LogisticRegression(fit_intercept=True, class_weight='balanced').fit(X, y)
    df_['response_pred'] = LR.predict(X)
    coef_ = LR.coef_
    df_['b1'] = coef_[0][0]
    df_['b2'] = coef_[0][1]
    df_['b0'] = LR.intercept_[0]

    # standardise the data
    X = StandardScaler().fit_transform(X)
    y = df_['response']
    LR = LogisticRegression(fit_intercept=True, class_weight='balanced').fit(X, y)
    df_['response_pred_stdz'] = LR.predict(X)
    coef_ = LR.coef_
    df_['b1_stdz'] = coef_[0][0]
    df_['b2_stdz'] = coef_[0][1]
    df_['b0_stdz'] = LR.intercept_[0]
        
        
    # Only use action plan position
    # X = StandardScaler().fit_transform(X)
    X = df_[['dist_mean']]
    y = df_['response']
    LR = LogisticRegression(fit_intercept=True, class_weight='balanced').fit(X, y)
    df_['response_pred_p_only'] = LR.predict(X)
    coef_ = LR.coef_
    df_['b1_stdz_p_only'] = coef_[0][0]
    # df_['b2_stdz'] = coef_[0][1]
    df_['b0_stdz_p_only'] = LR.intercept_[0]
                
    return df_

df = utils.GroupOperation.map(group, subjs)
df = pd.concat(df, axis=0)


df.head()

In [ ]:
fn = utils.Save.savepath('ana_one_dot_predicting_individual_beh_profile_cossmilarity', 'prediction.csv')
df.to_csv(fn, index=False)

In [ ]:
df_ = df.copy()
df_ = df_[['participant', 'actual control', 'angular bias', 'response_pred', 'group']]
df_ = df_.groupby(['participant', 'actual control', 'angular bias', 'group']).mean()
df_ = df_.reset_index()

grid = sns.FacetGrid(col='participant', col_wrap=5, data=df_)
grid.map_dataframe(sns.pointplot, 
                   x='actual control', 
                   y='response_pred', 
                   hue='angular bias')
grid.add_legend()


plt.figure()
sns.catplot(x='actual control', y='response_pred', hue='angular bias', data=df_, col='group', kind='point')


In [ ]:
df_ = df[['participant', 'group', 'b0', 'b1', 'b2', 'mu']].groupby(['participant', 'group']).mean()
df_ = df_.reset_index()
df_.plot()
df_.head()

## Correlation with reaching noise-free performance

Basically nothing happens

In [ ]:
df_ctrl = [utils.LoadData.behaviorData(subj, utils.ExpInfo.taskName[2]) for subj in subjs]
df_ctrl = pd.concat(df_ctrl, axis=0)
df_ctrl = df_ctrl.query('`actual control`==1')[['participant', 'group', 'number of reaching']]
df_ctrl = df_ctrl.groupby(['participant', 'group']).mean().reset_index()
df_m = df_.merge(df_ctrl, on=['participant', 'group'])
df_m['mu'] = -(df_m['b0'] / df_m['b1'])
df_m.head()

In [ ]:
print(df_m.corr())

# compute stats 
import pingouin as pg
print(pg.corr(df_m['b0'], df_m['b1'], method='pearson'))
print(pg.corr(df_m['b0'], df_m['b2'], method='pearson'))


In [ ]:
sns.regplot(x='b0', y='b1', data=df_m)

In [ ]:
sns.scatterplot(x='b0', y='b1', data=df_m, hue='group')

In [ ]:
# set figure size
plt.figure(figsize=(15, 10))
sns.barplot(x='participant', y='mu', data=df_m, hue='group')

In [ ]:
df_m.corr()

# individual

In [ ]:
subj = subjs[46]
df_beh = utils.LoadData.behaviorData(subj, task)

df_ = pd.concat([df_beh, pd.DataFrame(d)], axis=1)
X = df_[['dist_mean']]
X = df_[['v_mean']]
X = df_[['dist_mean', 'v_mean']]
# X = df_[['dist_mean', 'v_mean', 'dist_sd', 'v_sd']]
# X = df_[['dist_mean']]
X = StandardScaler().fit_transform(X)

y = df_['response']
# class_weightdict = {0: 1, 1: 1}
LR = LogisticRegression(fit_intercept=True, class_weight='balanced').fit(X, y)
df_['response_pred'] = LR.predict(X)
print(LR.coef_, LR.intercept_)

plt.figure()
sns.pointplot(x='actual control', y='response_pred', hue='angular bias', data=df_)

In [ ]:
d_ = pd.DataFrame(d)
df_ = pd.concat([df_beh, d_], axis=1)   
# df_['dist_mean']=1-df_['dist_mean']
plt.figure()
sns.pointplot(x='actual control', y='dist_mean', hue='angular bias', data=df_)
plt.figure()
sns.pointplot(x='actual control', y='v_mean', hue='angular bias', data=df_)
plt.figure()
sns.pointplot(x='actual control', y='dist_sd', hue='angular bias', data=df_)
plt.figure()
sns.pointplot(x='actual control', y='v_sd', hue='angular bias', data=df_)

In [ ]:
# n = 2
# mean = np.zeros(n)
# cov = np.eye(n)
# rv = multivariate_normal(mean=mean, cov=cov)
# samples = rv.rvs(10000)

# dist_sample = distance.cdist(np.expand_dims(mean, 0), samples, 'euclidean').flatten()
# sns.distplot(dist_sample)
# # plt.plot(samples[:, 0], samples[:, 1], 'o')

In [ ]:
def binary_separability_all(h_trial, h_disp_trial):
    """
    Compute the binary separability of the given dataset.
    """

    from sklearn.cluster import KMeans
    from sklearn.mixture import GaussianMixture
    # hidden action plan velocity
    vh = np.diff(h_trial, axis=0)
    vh_disp = np.diff(h_disp_trial, axis=0)

    Xs = {}

    X = np.vstack([h_trial, h_disp_trial])
    Xs['p'] = X
    X = np.vstack([vh, vh_disp])
    Xs['v'] = X


    hvh = np.hstack([h_trial[:-1], vh])
    hvh_disp = np.hstack([h_disp_trial[:-1], vh_disp])
    X = np.vstack([hvh, hvh_disp])
    Xs['p+v'] = X


    diff = h_disp_trial - h_trial
    diff_zero = np.zeros_like(diff)
    X = np.vstack([diff, diff_zero])
    Xs['p_diff'] = X


    diff = vh_disp - vh
    diff_zero = np.zeros_like(diff)
    X = np.vstack([diff, diff_zero])
    Xs['v_diff'] = X

    diff1 = h_disp_trial - h_trial
    diff2 = vh_disp - vh
    diff = np.hstack([diff1[:-1, :], diff2])
    diff_zero = np.zeros_like(diff)
    X = np.vstack([diff, diff_zero])
    Xs['p_diff+v_diff'] = X

    def binary_separability(X, nRun=10):
        labels = np.tile(np.array([0, 1]), [int(X.shape[0]/2), 1]).flatten(order='F')
        
        acc_run = []
        for iRun in range(nRun):
            # labels_ = KMeans(n_clusters=2, random_state=4).fit_predict(X)
            labels_ = GaussianMixture(n_components=2, random_state=iRun).fit_predict(X)
            
            acc = np.mean(labels_ == labels)
            if acc < 0.5:
                acc = 1-acc
            acc_run.append(acc)
        acc_run = np.array(acc_run).mean()
        return -acc

    for name, X in Xs.items():
        # print(name, binary_separability(X))
        Xs[name] = binary_separability(X)
    return Xs

# binary_separability_all(h_trial, h_disp_trial)

In [ ]:
# subj = subjs[5]
# iTrial = 0
wSize = 60


df_beh = utils.LoadData.behaviorData(subj, task)
_, h, _ = utils.LoadData.xhy(subj, task, wSize=wSize)
_, h_disp, _ = utils.LoadData.xhy_disp(subj, task, wSize=wSize)
dist_measure = 'euclidean'

# dist_timeSeries = []
# v_timeSeries = []
d = []
bs = []
for iTrial in range(len(h)):
    h_trial = h[iTrial][:-1, :]
    h_disp_trial = h_disp[iTrial]

    # hidden action plan velocity
    vh = np.diff(h_trial, axis=0)
    vh_disp_dot = np.diff(h_disp_trial, axis=0)

    dist_timeSeries_ = paired_distances(h_trial, h_disp_trial, metric=dist_measure)
    v_timeSeries_ = paired_distances(vh, vh_disp_dot, metric=dist_measure)
    
    d.append({'dist_mean': 1-dist_timeSeries_.mean(), 
              'dist_sd': -dist_timeSeries_.std(),
              'v_mean': 1-v_timeSeries_.mean(),
              'v_sd': -v_timeSeries_.std()})
    
    
    
    bs.append(binary_separability_all(h_trial, h_disp_trial))


In [ ]:
df_ = pd.concat([df_beh, pd.DataFrame(d), pd.DataFrame(bs)], axis=1)
columns = df_.loc[:, 'dist_mean':].columns.values
for column in columns:
    plt.figure()
    sns.pointplot(x='actual control', y=column, hue='angular bias', data=df_)
    plt.title(column)


In [ ]:
# df_ = pd.concat([df_beh, pd.DataFrame(d)], axis=1)
# X = df_[['p']]
# X = df_[['v']]
# X = df_[['p', 'v']]
X = df_[['p+v']]
X = df_[['p', 'v', 'p+v']]
# X = df_[['p_diff', 'v_diff']]
X = StandardScaler().fit_transform(X)

y = df_['response']
LR = LogisticRegression(fit_intercept=True, class_weight='balanced').fit(X, y)
df_['response_pred'] = LR.predict(X)
print(LR.coef_, LR.intercept_)

plt.figure()
sns.pointplot(x='actual control', y='response_pred', hue='angular bias', data=df_)

# By trial

In [ ]:
# subj = subjs[10]
# # iTrial = 0
# wSize = 60


# df_beh = utils.LoadData.behaviorData(subj, task)
# _, h, _ = utils.LoadData.xhy(subj, task, wSize=wSize)
# _, h_disp, _ = utils.LoadData.xhy_disp(subj, task, wSize=wSize)
# dist_measure = 'euclidean'


In [ ]:
# iTrial = 5
# h_trial = h[iTrial][:-1, :]
# h_disp_trial = h_disp[iTrial]

# # hidden action plan velocity
# vh = np.diff(h_trial, axis=0)
# vh_disp = np.diff(h_disp_trial, axis=0)

# dist_timeSeries_ = paired_distances(h_trial, h_disp_trial, metric=dist_measure)
# v_timeSeries_ = paired_distances(vh, vh_disp, metric=dist_measure)

# # d.append({'dist_mean': 1-dist_timeSeries_.mean(), 
# #             'dist_sd': -dist_timeSeries_.std(),
# #             'v_mean': 1-v_timeSeries_.mean(),
# #             'v_sd': -v_timeSeries_.std()})


    

In [ ]:
# print(df_beh.iloc[iTrial])


In [ ]:
# from sklearn.cluster import KMeans
# from sklearn.mixture import GaussianMixture
# Xs = {}

# X = np.vstack([h_trial, h_disp_trial])
# Xs['position'] = X
# X = np.vstack([vh, vh_disp])
# Xs['velocity'] = X


# hvh = np.hstack([h_trial[:-1], vh])
# hvh_disp = np.hstack([h_disp_trial[:-1], vh_disp])
# X = np.vstack([hvh, hvh_disp])
# Xs['p+v'] = X


# diff = h_disp_trial - h_trial
# diff_zero = np.zeros_like(diff)
# X = np.vstack([diff, diff_zero])
# Xs['p_diff'] = X


# diff = vh_disp - vh
# diff_zero = np.zeros_like(diff)
# X = np.vstack([diff, diff_zero])
# Xs['v_diff'] = X

# diff1 = h_disp_trial - h_trial
# diff2 = vh_disp - vh
# diff = np.hstack([diff1[:-1, :], diff2])
# diff_zero = np.zeros_like(diff)
# X = np.vstack([diff, diff_zero])
# Xs['p_diff+v_diff'] = X

# def binary_separability(X, nRun=10):
#     labels = np.tile(np.array([0, 1]), [int(X.shape[0]/2), 1]).flatten(order='F')
    
#     acc_run = []
#     for iRun in range(nRun):
#         # labels_ = KMeans(n_clusters=2, random_state=4).fit_predict(X)
#         labels_ = GaussianMixture(n_components=2, random_state=0).fit_predict(X)
        
#         acc = np.mean(labels_ == labels)
#         if acc < 0.5:
#             acc = 1-acc
#         acc_run.append(acc)
#     acc_run = np.array(acc_run).mean()
#     return acc

# for name, X in Xs.items():
#     print(name, binary_separability(X))


In [ ]:


# fig, ax = plt.subplots(figsize=(10, 5))
# ax.plot(dist_timeSeries_)
# ax.set_ylim([0, 2])
# plt.title('position dist')



# fig, ax = plt.subplots(figsize=(10, 5))
# ax.plot(v_timeSeries_)
# ax.set_ylim([0, 2])
# plt.title('v dist')


# # intention traj diff

# dd = utils.DataProcessing.diff(h_trial)
# fig, ax = plt.subplots(figsize=(10, 5))
# ax.plot(dd)
# ax.set_ylim([0, 2])
# plt.title('intention t t+1')

# dd = utils.DataProcessing.diff(h_disp_trial)
# fig, ax = plt.subplots(figsize=(10, 5))
# ax.plot(dd)
# ax.set_ylim([0, 2])
# plt.title('disp t t+1')

In [ ]:
# diff = h_disp_trial - h_trial
# mean = np.mean(diff, axis=0)
# cov = np.cov(diff, rowvar=0)

# from scipy.stats import multivariate_normal
# rv = multivariate_normal(mean=mean, cov=cov)
# nSample = 10000
# samples = rv.rvs(nSample)

In [ ]:
# from scipy.spatial import distance
# dist_sample = distance.cdist(np.expand_dims(mean, 0), samples, 'euclidean').flatten()
# # dist_sample = paired_distances(np.tile(mean, (nSample, 1)), samples, metric='euclidean')
# dist_target = distance.cdist(np.expand_dims(mean, 0), np.zeros((1, diff.shape[1])), 'euclidean')

In [ ]:
# plt.plot(dist_sample)
# plt.plot(dist_target, 'or')

In [ ]:
# n = 2
# mean = np.zeros(n)
# cov = np.eye(n)
# rv = multivariate_normal(mean=mean, cov=cov)
# samples = rv.rvs(10000)

# dist_sample = distance.cdist(np.expand_dims(mean, 0), samples, 'euclidean').flatten()
# sns.distplot(dist_sample)
# # plt.plot(samples[:, 0], samples[:, 1], 'o')